# Tensor HODMD Example with Binned Temperature Data

This notebook demonstrates the tensor Higher-Order Dynamic Mode Decomposition (HODMD) on the binned temperature data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from multi_iter_hodmd import MultiIterativeHODMD

## Load the Data

Load the binned temperature data from the numpy file.

In [ ]:
# Load the binned temperature data
data = np.load('binned_temperature_cs.npy')
print(f"Data shape: {data.shape}")
print(f"Data type: {data.dtype}")
print(f"Data range: [{data.min():.4f}, {data.max():.4f}]")

## Visualize the Data

Let's examine the structure of our tensor data.

In [ ]:
# Visualize a few time slices of the data
if len(data.shape) == 3:
    # Assume shape is (time, space1, space2)
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    time_indices = np.linspace(0, data.shape[0]-1, 6, dtype=int)
    
    for i, t_idx in enumerate(time_indices):
        im = axes[i].imshow(data[t_idx], cmap='viridis', aspect='auto')
        axes[i].set_title(f'Time index {t_idx}')
        axes[i].set_xlabel('Space dimension 2')
        axes[i].set_ylabel('Space dimension 1')
        plt.colorbar(im, ax=axes[i])
    
    plt.tight_layout()
    plt.show()
elif len(data.shape) == 4:
    # Assume shape is (time, space1, space2, space3)
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    time_indices = np.linspace(0, data.shape[0]-1, 6, dtype=int)
    mid_slice = data.shape[3] // 2  # Middle slice in the 3rd spatial dimension
    
    for i, t_idx in enumerate(time_indices):
        im = axes[i].imshow(data[t_idx, :, :, mid_slice], cmap='viridis', aspect='auto')
        axes[i].set_title(f'Time {t_idx}, z-slice {mid_slice}')
        axes[i].set_xlabel('Space dimension 2')
        axes[i].set_ylabel('Space dimension 1')
        plt.colorbar(im, ax=axes[i])
    
    plt.tight_layout()
    plt.show()

## Initialize Tensor HODMD

Set up the Tensor HODMD parameters and initialize the decomposition.

In [ ]:
# Set parameters for Tensor HODMD
d = 5  # Number of delays
rank = [10, 10, 10]  # Tucker decomposition ranks (adjust based on your data dimensions)
dt = 1.0  # Time step

# Adjust rank based on actual data dimensions
if len(data.shape) == 3:
    rank = [min(10, data.shape[1]), min(10, data.shape[2])]  # For 2D spatial data
elif len(data.shape) == 4:
    rank = [min(10, data.shape[1]), min(10, data.shape[2]), min(10, data.shape[3])]  # For 3D spatial data

print(f"Using delay d = {d}")
print(f"Using Tucker ranks: {rank}")
print(f"Time step dt = {dt}")

In [ ]:
# Initialize Multi-Iterative HODMD
tensor_hodmd = MultiIterativeHODMD(d=d, rank=rank, dt=dt)
print("Multi-Iterative HODMD initialized successfully")

## Fit the Model

Fit the Tensor HODMD model to our data.

In [ ]:
# Fit the model
print("Fitting Tensor HODMD model...")
tensor_hodmd.fit(data)
print("Model fitted successfully!")

## Analyze Results

Examine the eigenvalues and modes from the decomposition.

In [ ]:
# Get eigenvalues and analyze them
eigenvalues = tensor_hodmd.eigenvalues
print(f"Number of eigenvalues: {len(eigenvalues)}")

# Plot eigenvalues in the complex plane
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.scatter(eigenvalues.real, eigenvalues.imag, alpha=0.7)
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Eigenvalues in Complex Plane')
plt.grid(True, alpha=0.3)

# Plot magnitude and phase
plt.subplot(1, 2, 2)
magnitudes = np.abs(eigenvalues)
plt.hist(magnitudes, bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('Eigenvalue Magnitude')
plt.ylabel('Count')
plt.title('Distribution of Eigenvalue Magnitudes')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Eigenvalue magnitudes range: [{magnitudes.min():.4f}, {magnitudes.max():.4f}]")

## Compute Frequencies

Extract the frequencies from the eigenvalues.

In [ ]:
# Compute frequencies
frequencies = tensor_hodmd.frequency

# Plot frequency spectrum
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.stem(np.arange(len(frequencies)), frequencies.real, basefmt=" ")
plt.xlabel('Mode Index')
plt.ylabel('Real Frequency')
plt.title('Real Part of Frequencies')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.stem(np.arange(len(frequencies)), frequencies.imag, basefmt=" ")
plt.xlabel('Mode Index')
plt.ylabel('Imaginary Frequency')
plt.title('Imaginary Part of Frequencies')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Number of frequencies: {len(frequencies)}")
print(f"Frequency range (real): [{frequencies.real.min():.4f}, {frequencies.real.max():.4f}]")
print(f"Frequency range (imag): [{frequencies.imag.min():.4f}, {frequencies.imag.max():.4f}]")

## Reconstruct and Predict

Use the fitted model to reconstruct the original data and make predictions.

In [ ]:
# Reconstruct the original data
print("Reconstructing data...")
reconstructed = tensor_hodmd.reconstructed_data
print(f"Reconstruction shape: {reconstructed.shape}")

# Compute reconstruction error
# Note: reconstructed data might have different time dimension due to delay embedding
min_time = min(data.shape[0], reconstructed.shape[0])
error = np.linalg.norm(data[:min_time] - reconstructed[:min_time]) / np.linalg.norm(data[:min_time])
print(f"Relative reconstruction error: {error:.6f}")

In [ ]:
# Predict future time steps
n_predict = 10
print(f"Predicting {n_predict} future time steps...")

# Create time array for prediction
future_times = np.arange(data.shape[0], data.shape[0] + n_predict) * dt
predicted = tensor_hodmd.predict(future_times)
print(f"Prediction shape: {predicted.shape}")

## Visualize Results

Compare original, reconstructed, and predicted data.

In [ ]:
# Visualize comparison for 2D or 3D data
if len(data.shape) == 3:
    # For 3D tensors (time, space1, space2)
    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    
    time_indices = [0, data.shape[0]//2, min_time-1]
    titles = ['Early Time', 'Middle Time', 'Late Time']
    
    for i, (t_idx, title) in enumerate(zip(time_indices, titles)):
        # Original
        im1 = axes[0, i].imshow(data[t_idx], cmap='viridis', aspect='auto')
        axes[0, i].set_title(f'Original - {title}')
        plt.colorbar(im1, ax=axes[0, i])
        
        # Reconstructed
        im2 = axes[1, i].imshow(reconstructed[t_idx], cmap='viridis', aspect='auto')
        axes[1, i].set_title(f'Reconstructed - {title}')
        plt.colorbar(im2, ax=axes[1, i])
        
        # Error
        error_slice = data[t_idx] - reconstructed[t_idx]
        im3 = axes[2, i].imshow(error_slice, cmap='RdBu', aspect='auto')
        axes[2, i].set_title(f'Error - {title}')
        plt.colorbar(im3, ax=axes[2, i])
    
    plt.tight_layout()
    plt.show()

elif len(data.shape) == 4:
    # For 4D tensors (time, space1, space2, space3) - show middle z-slice
    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    
    time_indices = [0, data.shape[0]//2, min_time-1]
    titles = ['Early Time', 'Middle Time', 'Late Time']
    mid_z = data.shape[3] // 2
    
    for i, (t_idx, title) in enumerate(zip(time_indices, titles)):
        # Original
        im1 = axes[0, i].imshow(data[t_idx, :, :, mid_z], cmap='viridis', aspect='auto')
        axes[0, i].set_title(f'Original - {title}')
        plt.colorbar(im1, ax=axes[0, i])
        
        # Reconstructed
        im2 = axes[1, i].imshow(reconstructed[t_idx, :, :, mid_z], cmap='viridis', aspect='auto')
        axes[1, i].set_title(f'Reconstructed - {title}')
        plt.colorbar(im2, ax=axes[1, i])
        
        # Error
        error_slice = data[t_idx, :, :, mid_z] - reconstructed[t_idx, :, :, mid_z]
        im3 = axes[2, i].imshow(error_slice, cmap='RdBu', aspect='auto')
        axes[2, i].set_title(f'Error - {title}')
        plt.colorbar(im3, ax=axes[2, i])
    
    plt.tight_layout()
    plt.show()

## Visualize Predicted Data

Show the predicted future time steps.

In [ ]:
# Visualize predictions
if predicted is not None:
    if len(predicted.shape) == 3:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        pred_indices = np.linspace(0, predicted.shape[0]-1, 6, dtype=int)
        
        for i, p_idx in enumerate(pred_indices):
            im = axes[i].imshow(predicted[p_idx], cmap='viridis', aspect='auto')
            axes[i].set_title(f'Predicted step {p_idx+1}')
            axes[i].set_xlabel('Space dimension 2')
            axes[i].set_ylabel('Space dimension 1')
            plt.colorbar(im, ax=axes[i])
        
        plt.suptitle('Predicted Future Time Steps')
        plt.tight_layout()
        plt.show()
    
    elif len(predicted.shape) == 4:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        pred_indices = np.linspace(0, predicted.shape[0]-1, 6, dtype=int)
        mid_z = predicted.shape[3] // 2
        
        for i, p_idx in enumerate(pred_indices):
            im = axes[i].imshow(predicted[p_idx, :, :, mid_z], cmap='viridis', aspect='auto')
            axes[i].set_title(f'Predicted step {p_idx+1} (z={mid_z})')
            axes[i].set_xlabel('Space dimension 2')
            axes[i].set_ylabel('Space dimension 1')
            plt.colorbar(im, ax=axes[i])
        
        plt.suptitle('Predicted Future Time Steps')
        plt.tight_layout()
        plt.show()

## Summary

Display a summary of the analysis results.

In [ ]:
print("=== Tensor HODMD Analysis Summary ===")
print(f"Data shape: {data.shape}")
print(f"Number of delays: {d}")
print(f"Tucker decomposition ranks: {rank}")
print(f"Number of modes: {len(eigenvalues)}")
print(f"Reconstruction error: {error:.6f}")
print(f"Predicted {n_predict} future time steps")

# Find most significant modes (largest magnitude eigenvalues)
significant_modes = np.argsort(np.abs(eigenvalues))[-5:]
print("\nTop 5 most significant modes:")
for i, mode_idx in enumerate(reversed(significant_modes)):
    eigenval = eigenvalues[mode_idx]
    freq = frequencies[mode_idx]
    print(f"  Mode {mode_idx}: λ = {eigenval:.4f}, f = {freq:.4f}, |λ| = {np.abs(eigenval):.4f}")